In [1]:
!git clone https://github.com/baoguangsheng/fast-detect-gpt.git

Cloning into 'fast-detect-gpt'...
remote: Enumerating objects: 647, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 647 (delta 79), reused 86 (delta 71), pack-reused 545 (from 1)
Receiving objects: 100% (647/647), 228.84 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (476/476), done.
Updating files: 100% (509/509), done.


In [2]:
%cd fast-detect-gpt/scripts

/kaggle/working/fast-detect-gpt/scripts


In [3]:
!bash setup.sh

bash: setup.sh: No such file or directory


In [4]:
import pandas as pd

df = pd.read_csv('/kaggle/input/generated-text/data_split/test.csv')
df.head()

,id,prompt_id,text,generated
0,e_8tt3v1l1,8,Summer projects should be student basedecausei...,1
1,e_yu4mige3,4,"My position on driverless cars is, that they w...",0
2,e_cmiw5i1g,5,I'm against to the use of this technology to r...,1
3,e_sdxf4v6a,8,Summer projects should be studentdesigned beca...,1
4,e_d4wsemmz,10,"Mom,"" Generic_Name whimpered sadly. ""Why is it...",1


In [5]:
# Copyright (c) Guangsheng Bao.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
import random

import numpy as np
import torch
import os
import glob
import argparse
import json
from model import load_tokenizer, load_model
from fast_detect_gpt import get_sampling_discrepancy_analytic

results = []

# estimate the probability according to the distribution of our test results on ChatGPT and GPT-4
class ProbEstimator:
    def __init__(self, args):
        self.real_crits = []
        self.fake_crits = []
        for result_file in glob.glob(os.path.join(args.ref_path, '*.json')):
            with open(result_file, 'r') as fin:
                res = json.load(fin)
                self.real_crits.extend(res['predictions']['real'])
                self.fake_crits.extend(res['predictions']['samples'])
        print(f'ProbEstimator: total {len(self.real_crits) * 2} samples.')


    def crit_to_prob(self, crit):
        offset = np.sort(np.abs(np.array(self.real_crits + self.fake_crits) - crit))[100]
        cnt_real = np.sum((np.array(self.real_crits) > crit - offset) & (np.array(self.real_crits) < crit + offset))
        cnt_fake = np.sum((np.array(self.fake_crits) > crit - offset) & (np.array(self.fake_crits) < crit + offset))
        return cnt_fake / (cnt_real + cnt_fake)

# run interactive local inference
def run(args):
    # load model
    scoring_tokenizer = load_tokenizer(args.scoring_model_name, args.dataset, args.cache_dir)
    scoring_model = load_model(args.scoring_model_name, args.device, args.cache_dir)
    scoring_model.eval()
    if args.reference_model_name != args.scoring_model_name:
        reference_tokenizer = load_tokenizer(args.reference_model_name, args.dataset, args.cache_dir)
        reference_model = load_model(args.reference_model_name, args.device, args.cache_dir)
        reference_model.eval()
    # evaluate criterion
    name = "sampling_discrepancy_analytic"
    criterion_fn = get_sampling_discrepancy_analytic
    prob_estimator = ProbEstimator(args)
    # input text
    for index, row in df.iterrows():
        text = row['text']
        if len(text) == 0:
            break
        # evaluate text
        tokenized = scoring_tokenizer(text, truncation=True, return_tensors="pt", padding=True, return_token_type_ids=False).to(args.device)
        labels = tokenized.input_ids[:, 1:]
        with torch.no_grad():
            logits_score = scoring_model(**tokenized).logits[:, :-1]
            if args.reference_model_name == args.scoring_model_name:
                logits_ref = logits_score
            else:
                tokenized = reference_tokenizer(text, truncation=True, return_tensors="pt", padding=True, return_token_type_ids=False).to(args.device)
                assert torch.all(tokenized.input_ids[:, 1:] == labels), "Tokenizer is mismatch."
                logits_ref = reference_model(**tokenized).logits[:, :-1]
            crit = criterion_fn(logits_ref, logits_score, labels)
        # estimate the probability of machine generated text
        prob = prob_estimator.crit_to_prob(crit)
        print(f'Fast-DetectGPT criterion is {crit:.4f}, suggesting that the text has a probability of {prob * 100:.0f}% to be machine-generated.')
        print()
        results.append({
        'id': row['id'],
        'criterion': crit,
        'probability': prob
        })
    results_df = pd.DataFrame(results)
    updated_df = pd.merge(df, results_df[['id', 'criterion', 'probability']], on='id')
    updated_df.to_csv('/kaggle/working/test_content.csv', index=False)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--reference_model_name', type=str, default="gpt-neo-2.7B")  # use gpt-j-6B for more accurate detection
    parser.add_argument('--scoring_model_name', type=str, default="gpt-neo-2.7B")
    parser.add_argument('--dataset', type=str, default="xsum")
    parser.add_argument('--ref_path', type=str, default="../local_infer_ref")
    parser.add_argument('--device', type=str, default="cuda")
    parser.add_argument('--cache_dir', type=str, default="../cache")
    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    run(args)

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading model EleutherAI/gpt-neo-2.7B...


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

Moving model to GPU...DONE (3.48s)
ProbEstimator: total 1800 samples.
Fast-DetectGPT criterion is 0.8697, suggesting that the text has a probability of 27% to be machine-generated.

Fast-DetectGPT criterion is 0.9002, suggesting that the text has a probability of 32% to be machine-generated.

Fast-DetectGPT criterion is 0.7110, suggesting that the text has a probability of 22% to be machine-generated.

Fast-DetectGPT criterion is 6.2725, suggesting that the text has a probability of 100% to be machine-generated.

Fast-DetectGPT criterion is -2.8760, suggesting that the text has a probability of 0% to be machine-generated.

Fast-DetectGPT criterion is 4.5367, suggesting that the text has a probability of 100% to be machine-generated.

Fast-DetectGPT criterion is 5.0201, suggesting that the text has a probability of 100% to be machine-generated.

Fast-DetectGPT criterion is -0.5239, suggesting that the text has a probability of 2% to be machine-generated.

Fast-DetectGPT criterion is 7.4